# PetFinder.my

In [ ]:
import os
import pickle

import keras.utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications import Xception
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Activation, Flatten, Dense
from keras.layers import (concatenate)
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

train_image = os.path.join(data_dir, "train_images")
test_image = os.path.join(data_dir, "test_images")

label_column = "AdoptionSpeed"

BATCH_SIZE, test_size = 512, 0.2

height, width = 100, 100

# Data processing
### Helper functions for data processing

In [ ]:
def read_csv_kaggle(path, is_train):
    train = pd.read_csv(path, sep=',')
    pet_ids = train["PetID"]

    selected_columns = ["Type",
                        "Gender",
                        "Color1",
                        "Color2",
                        "Color3",
                        "MaturitySize",
                        "FurLength",
                        "Vaccinated",
                        "Dewormed",
                        "Sterilized",
                        "Health",
                        #"State",
                        "MaturitySize"]
    
    y = train[label_column] if is_train else None
    
    # One-Hot-encode
    X = pd.get_dummies(train[selected_columns], columns=selected_columns)

    # Normalize:
    to_normalize = ["Age", "Fee", "Quantity"]
    for to_norm in to_normalize:
         X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()
    
    return X, y, pet_ids

def read_images(image_paths):
    def random_image():
        return np.random.rand(height, width, 3) * 255
    
    def read_image(path):
        img = np.asarray(Image.open(path).convert("RGB"), dtype="int32" )
        return resize(img, (height, width), anti_aliasing=True, mode='constant')    

    return np.array([read_image(path) if os.path.isfile(path) else random_image() for path in tqdm(image_paths)])

### Read training CSV

In [ ]:
X, y, pet_ids = read_csv_kaggle(os.path.join(data_dir, "train.csv"), True)

### Read training images

In [ ]:
f_im_name = "images.binary"

if not os.path.isfile(f_im_name):
    images = read_images([os.path.join(train_image, pet_id + "-1.jpg") for pet_id in pet_ids])
    
    # Standardize:
    mean, std = np.mean(images), np.std(images)
    images_meanstd = (images - mean)/std
    
    with open(f_im_name, 'wb') as handle_1, open("mnstd", 'wb') as handle_2:
        pickle.dump(images_meanstd, handle_1, protocol=pickle.HIGHEST_PROTOCOL)        
        pickle.dump((mean, std), handle_2, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f_im_name, 'rb') as handle_1, open("mnstd", 'rb') as handle_2:
        images_meanstd = pickle.load(handle_1)
        temp = pickle.load(handle_2)
        mean, std = temp[0], temp[1]
                
print(images_meanstd.shape)

### Split traint/test

In [ ]:
X_train_img, X_test_img, X_train_else, X_test_else, y_train, y_test = train_test_split(images_meanstd, 
                                                                                       X, 
                                                                                       y, 
                                                                                       test_size=test_size,
                                                                                       random_state=1)

# Create & train models

### Baseline model

In [ ]:
rf = RandomForestClassifier(n_estimators=65, max_depth=15)
rf.fit(X_train_else, y_train)

### Neural Net solution

In [ ]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

transfer = Xception(weights='imagenet', include_top=False, input_shape=(height, width, 3))

# Freeze Xception
#for layer in transfer.layers[:-9]:
#    layer.trainable = False

# Inputs
image_input = Input(shape=(height, width, 3))
aux_input = Input(shape=(len(list(X_train_else)),))

# Images:
transfer = transfer(image_input)
transfer = Dense(150, activation='relu')(transfer)
flatten = Flatten()(transfer)

# Aux input:
x = Dense(150, activation='relu')(aux_input)

# Merged:
merge = concatenate([flatten, x])
x = Dense(500)(merge)
h = Dense(5)(x)

# Predictions:
predictions = Activation('softmax')(h)

model = Model(inputs=[image_input, aux_input], outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit([X_train_img, X_train_else], 
                    keras.utils.to_categorical(y_train),
                    batch_size=BATCH_SIZE, 
                    epochs=4, 
                    validation_split=0.1,
                    callbacks=[ModelCheckpoint('test_model.h5', save_best_only=True)])

In [ ]:
plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'])
plt.title('Accuracy')

# Evaluate models

## Neural Net

In [ ]:
train_pred = [np.argmax(pred) for pred in model.predict([X_train_img, X_train_else])]
test_predictions = [np.argmax(pred) for pred in model.predict([X_test_img, X_test_else])]

In [ ]:
print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

## Baseline

In [ ]:
train_pred = [pred for pred in rf.predict(X_train_else)]
test_predictions = [pred for pred in rf.predict(X_test_else)]

print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

## Kaggle Test

In [ ]:
test, _, test_pet_ids = read_csv_kaggle(os.path.join(data_dir, "test.csv"), False)
test_images = read_images([os.path.join(test_image, pet_id + "-1.jpg") for pet_id in test_pet_ids])
test_images_std = (test_images - mean)/std

### Neural Net

In [ ]:
loaded = load_model('test_model.h5')
test_pred = loaded.predict([test_images_std, test])
test_pred_label = [np.argmax(pred) for pred in test_pred]
pd.DataFrame({'PetID': test_pet_ids, 'AdoptionSpeed': test_pred_label}).to_csv('neural.csv', index=False)

### Baseline

In [ ]:
pd.DataFrame({'PetID': test_pet_ids, 'AdoptionSpeed': rf.predict(test)}).to_csv('baseline.csv', index=False)